In [1]:
import glob

In [2]:
pwd

'/work/albertl_uri_edu/fluxtoflow/nldas'

In [22]:
x = glob.glob('clipped/*.nc')

In [4]:
len(x)

2617

In [5]:
import os

In [10]:
import numpy as np

In [23]:
y = np.asarray([os.path.getsize(i)*1e-6 for i in x]) #megabytes
y.shape

(2617,)

In [12]:
import matplotlib.pyplot as plt

In [24]:
print(f'mean: {np.mean(y)}\n'
      f'max: {np.max(y)}\n'
      f'min: {np.min(y)}\n'
      f'sum: {np.sum(y)}'
     )


#strange that the numbers cut off differently.

mean: 0.40222499999999994
max: 0.402225
min: 0.402225
sum: 1052.622825


In [21]:
.4 * 2617

1046.8

In [26]:
import xarray as xr

/work/albertl_uri_edu/.conda/envs/sm/lib/python3.9/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [36]:
import torch

def nc2torch(i):
    return torch.from_numpy(np.asarray(i)).unsqueeze(0)

xx = xr.open_dataset(x[0])

x = glob.glob('clipped/*.nc')
dset = torch.empty([0,2,xx.Qs_summed.shape[0],xx.Qs_summed.shape[1]])

for i in sorted(x):
    j = xr.open_dataset(i)
    z0 = nc2torch(j.Qs_summed)
    z1 = nc2torch(j.Qsb_summed)
    z2 = torch.cat((z0,z1),axis=0).unsqueeze(0)
    dset = torch.cat((dset,z2),axis=0)
print(dset.shape)


torch.Size([2617, 2, 167, 288])


In [37]:
torch.save(dset,'nldas_ms.pt')

In [31]:
z0 = xr.open_dataset(x[0])
z1 = nc2torch(z0.Qs_summed)
z2 = nc2torch(z0.Qsb_summed)
# z1.shape
z3 = torch.cat((z1,z2),axis=0)
z3.shape

torch.Size([2, 167, 288])

In [70]:
y = []
with open('ms_streamflow_010115__030122.txt','r') as f:
    for x in f:
        y.append(x)
y = np.asarray(y)
print('number of lines in text file',y.shape)

streamflows = y[30:] #gets rid of meta data 
print(streamflows.shape)
print(streamflows[0])

## removes tabs, splits words into chunks
z = []
for idx,x in enumerate(streamflows):
    y = x.split('\t')
    z.append(y)
z = np.asarray(z)


#creates table with labels
a = np.expand_dims(np.asarray(['Gov Ent','Gage ID','Acq Date','cubic ft/s','unknown']),0)
b = np.vstack((a,z))
# b = b[:-5] #gets rid of dates we don't want
# print(b[1],'\n',b[-1])

#takes column 4 only (discharge) and replaces numbers that aren't integers with dummy number.
cfs = b[1:,3]
d = np.empty((0))
for x in cfs:
    try:
        d = np.append(d,int(x))
    except:
        print('+1 empty, replaced with -12345678')
        d = np.append(d,-12345678)

#grabs date column        
dte = b[1:,2]

##turns cubic feet per second to tensor for training with larsonet.
cfs0 = torch.from_numpy(np.float32(cfs)).unsqueeze(1).unsqueeze(1).unsqueeze(1)
torch.save(cfs0,'ms_cfs.pt')
np.save('ms_dte.npy',dte)

number of lines in text file (2647,)
(2617,)
USGS	07374000	2015-01-01	345000	A



In [71]:
type(cfs[0])

cfs0 = torch.from_numpy(np.float32(cfs)).unsqueeze(1).unsqueeze(1).unsqueeze(1)
torch.save(cfs0,'ms_cfs.pt')
np.save('ms_dte.npy',dte)


###### can't do this, pytorch doesn't like date/times 
###### dte0 = np.float32(dte)

In [44]:
x = glob.glob('clipped/*.nc')
print(x[0])
print(x[-1])
print(len(x))

clipped/NLDAS_VIC0125_H.A20150101_ms_clipped.nc
clipped/NLDAS_VIC0125_H.A20220301_ms_clipped.nc
2617
